# Mergesort

```
> ;;; p(4 1 3 2) -> \\
> ;;; p(4 1) p(3 2) m -> \\
> ;;; p(4) p(1) m p(3 2) m -> \\
> ;;; s(4) s(1) m p(3 2) m -> \\
> ;;; s(1 4) p(3 2) m -> \\
> ;;; s(1 4) p(3) p(2) m m -> \\
> ;;; s(1 4) s(3) s(2) m m -> \\
> ;;; s(1 4) s(2 3) m -> \\
> ;;; s(1 2 3 4) \\
>
>
> ;;; The state of the system consists of a list ls and an index into the \\
> ;;; list.  The elements of the list are either tagged as problems (p) \\
> ;;; solutions (s) or the symbol m (to suggest a merge). \\
> ;;; The program map locates a redex (a tagged sublist or 'm) and \\
> ;;; reduces the list based on the following rules: \\
>
> ;;; reduce-split:  [(... (p . seq) ...), i] => \\
> ;;;                      ^i
> ;;; [( ... (p . seq1) (p . seq2) m ....), i] \\
> ;;;        ^i
>
> ;;; reduce-promote: [(... (p n) ....), i] => \\
> ;;;                       ^i
> ;;; [(... (s n) ...), i+1] \\
>
>
> ;;; reduce-merge: [(... (s . seq1) (s . seq2) m ...), i] => \\
> ;;;                                           ^i
> ;;; [(... (s . (merge seq1 seq2)) ....), i-1] \\
>
>
> ;;; [((p 4 1 3 2)), 0] ->split \\
> ;;; [((p 4 1) (p 3 2) m), 0] ->split \\
> ;;; [((p 4) (p 1) m  (p 3 2) m), 0] ->promote \\
> ;;; [((s 4) (p 1) m  (p 3 2) m), 1] ->promote \\
> ;;; [((s 4) (s 1) m  (p 3 2) m), 2] ->merge \\
> ;;; [((s 1 4) (p 3 2) m), 1] ->split \\
> ;;; [((s 1 4) (p 3) (p 2) m m), 1] ->promote \\
> ;;; [((s 1 4) (s 3) (p 3) m m), 2] ->promote \\
> ;;; [((s 1 4) (s 3) (s 2) m m), 3] ->merge \\
> ;;; [((s 1 4) (s 1 3) m), 2] ->merge \\
> ;;; [((s 1 2 3 4)) 1] fixed point \\
```

In [ ]:
from mapcode.mapcode import mapcode
from merge import merge
import math
from functools import reduce
from collections import deque
from typing import List, Tuple, Any, Callable, Dict

## Utility Functions

In [ ]:
def split_half(lst: List[Any]) -> Tuple[List[Any], List[Any]]:
    """Split list into two halves."""
    mid = len(lst) // 2
    return lst[:mid], lst[mid:]


PROBLEM_TAG = "p"
SORTED_TAG = "s"
MERGE_TAG = "m"


def is_problem(p):
    return isinstance(p, list) and p and p[0] == PROBLEM_TAG


def problem_list(p):
    return p[1:]


def problem(ls):
    return [PROBLEM_TAG] + ls


def is_sorted(s):
    return isinstance(s, list) and s and s[0] == SORTED_TAG


def sorted_list(s):
    return s[1:]


def sorted_tag(ls):
    return [SORTED_TAG] + ls


assert is_problem(problem([3, 1, 2]))
assert is_problem(problem([]))
assert is_problem(problem([4]))
assert not is_problem([3, 1, 2])
assert is_sorted(sorted_tag([3, 1, 2]))
assert is_sorted(sorted_tag([]))
assert is_sorted(sorted_tag([4]))
assert not is_sorted([3, 1, 2])


def promotable(state):
    ls, i = state
    return is_problem(ls[i]) and len(problem_list(ls[i])) == 1


def merge(left: List[int], right: List[int]) -> List[int]:
    """Merge two sorted lists."""
    result = []
    i = j = 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    result.extend(left[i:])
    result.extend(right[j:])
    return result


def strict(f: Callable) -> Callable:
    """Make function strict by evaluating its arguments."""
    return lambda *args: f(*args)


def promotable(state):
    ls, i = state
    return is_problem(ls[i]) and len(problem_list(ls[i])) == 1


def promote_transform(p):
    return ["s"] + p[1:]


def promote_reduce(ls, i):
    return list_graft(ls, i, i, [promote_transform(ls[i])])


def splittable(state):
    ls, i = state
    return is_problem(ls[i]) and len(ls[i]) > 2


def split_transform(p):
    left, right = split_half(problem_list(p))
    return [problem(left), problem(right), "m"]


def split_reduce(ls, i):
    return list_graft(ls, i, i, split_transform(ls[i]))


def mergeable(state):
    ls, i = state
    if i < 2 or i >= len(ls):
        return False
    return ls[i] == "m" and is_sorted(ls[i - 1]) and is_sorted(ls[i - 2])


def merge_reduce(ls, i):
    left = sorted_list(ls[i - 2])
    right = sorted_list(ls[i - 1])
    merged = sorted_tag(merge(left, right))
    return ls[: i - 2] + [merged] + ls[i + 1 :]


def done(state):
    ls, i = state
    return i == 0 and len(ls) == 1 and is_sorted(ls[0])


def advanceable(state):
    ls, i = state
    return i < len(ls) and is_sorted(ls[i])


def bot() -> Any:
    return []


def list_graft(lst: List[Any], start: int, end: int, repl: List[Any]) -> List[Any]:
    """Replace a slice of the list with another list."""
    return lst[:start] + repl + lst[end + 1 :]


def make_hash(pairs: List[Tuple[Any, Any]]) -> Dict[Any, Any]:
    return dict(pairs)

## Mapcode Style Mergesort

In [ ]:
def rho_mapcode(ls):
    return [[problem(ls)], 0]


def F_mapcode(state):
    ls, i = state
    if done(state):
        return state
    if advanceable(state):
        return [ls, i + 1]
    if mergeable(state):
        return [merge_reduce(ls, i), i - 2]
    if splittable(state):
        return [split_reduce(ls, i), i]
    if promotable(state):
        return [promote_reduce(ls, i), i]
    return state


def pi_mapcode(state):
    return state[0][0][1:]


mergesort_mapcode = mapcode(rho_mapcode, F_mapcode, pi_mapcode)

## Recursive Mergesort via Fixed Points

In [ ]:
def singleton(ls):
    return isinstance(ls, list) and len(ls) == 1


def trivial(p):
    return p == [] or singleton(p)


smerge = strict(merge)


def rho_H(ls):
    return [[ls], {}]


def H_fn(state):
    problems, fns = state
    if not problems:
        return state
    problem, *tl = problems
    if trivial(problem):
        fns[tuple(problem)] = lambda x: problem
        return [tl, fns]
    left, right = split_half(problem)
    fns[tuple(problem)] = lambda x: smerge(x[tuple(left)], x[tuple(right)])
    return [[left, right] + tl, fns]


def pi_H(state):
    return state[1]


def gen_fns(ls):
    return mapcode(rho_H, H_fn, pi_H)(ls)


def apply_fns(
    fns: Dict[Tuple[int], Callable], x: Dict[Tuple[int], List[int]]
) -> Dict[Tuple[int], List[int]]:
    return {k: fns[k](x) for k in x}


def bot_hash(keys: List[Tuple[int]]) -> Dict[Tuple[int], List[int]]:
    return {k: bot() for k in keys}


def rho_next(fns):
    return [lambda x: apply_fns(fns, x), bot_hash(list(fns.keys()))]


def next_fn(state):
    G, x = state
    return [G, G(x)]


def pi_next(state):
    return state[1]


def mergesort_star(ls):
    fns = gen_fns(ls)
    return mapcode(rho_next, next_fn, pi_next)(fns)


def mergesort_rec_star(ls):
    x_star = mergesort_star(ls)
    return x_star[tuple(ls)]

In [ ]:
def test_mergesort(mergesort_fn):
    assert mergesort_fn([]) == []
    assert mergesort_fn([3]) == [3]
    assert mergesort_fn([1, 3]) == [1, 3]
    assert mergesort_fn([3, 1]) == [1, 3]
    assert mergesort_fn([1, 3, 2]) == [1, 2, 3]
    assert mergesort_fn([3, 2, 1]) == [1, 2, 3]
    assert mergesort_fn([3, 2, 1, 4]) == [1, 2, 3, 4]


print("Testing mergesort with mapcode...", end=" ")
test_mergesort(mergesort_mapcode)
print("done.")
print("Testing mergesort with recursive star...", end=" ")
test_mergesort(mergesort_rec_star)
print("done.")